In [11]:
import ipynb.fs #Boilerplate for import ipynb.fs #Boilerplate for ipynb botebooks as modules

from utils import facenet
import math
from utils import lfw, tf_graph_viz
# from .defs.Facenet_impl import get_face_embeddings
import os, sys
import numpy as np
import tensorflow as tf
from sklearn import metrics
from scipy.optimize import brentq
from scipy import interpolate

In [2]:
# Embeddings are calculated using the pairs from http://vis-www.cs.umass.edu/lfw/pairs.txt
pairs_path = '/home/prudhvi/Desktop/Distillation/mobilenet-facenet-distillation/data/pairs.txt'
pairs = lfw.read_pairs(pairs_path)

In [3]:
dataset_path = "~/Desktop/Datasets/lfw/lfw_mtcnnpy_160"
# Get the paths for the corresponding images
paths, actual_issame = lfw.get_paths(os.path.expanduser(dataset_path), pairs)

In [14]:
def get_face_embeddings(data, model_path = "/home/prudhvi/Desktop/Models/tf/Facenet/20180402-114759", image_size = 160, batch_size = 64, save_to_disk = False, save_dir = None, mode = "dataset"):
    
    if save_to_disk:
        print("Saving to disk!")
    
    if mode == 'dataset':
        # Check that there are at least one training image per class
        for cls in data:
            assert(len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset')
        
        # Extract and show data stats
        paths, labels = facenet.get_image_paths_and_labels(data)
        print('Number of classes: %d' % len(data))
        print('Number of images: %d' % len(paths))
    elif mode == "paths":
        paths = data
    else:
        print("Error! Wrong mode specified")
        return
    
    with tf.Graph().as_default():
      
        with tf.Session() as sess:
            
            np.random.seed(seed=1)
            
            # Load the model
            print('Loading feature extraction model')
            facenet.load_model(model_path)
            
            # Get input and output tensors
            images_placeholder = tf.get_default_graph().get_tensor_by_name("input_6_2:0")
            embeddings = tf.get_default_graph().get_tensor_by_name("lambda_6_1/l2_normalize:0")
            embedding_size = embeddings.get_shape()[1]
            print("embedding size", embedding_size)
            
             # Run forward pass to calculate embeddings
            emb_dict={}
            print('Calculating features for images')
            nrof_images = len(paths)
            nrof_batches_per_epoch = int(math.ceil(1.0*nrof_images / batch_size))
            emb_array = np.zeros((nrof_images, embedding_size))
            print("Number of epochs", nrof_batches_per_epoch)
            for i in range(nrof_batches_per_epoch):
                start_index = i*batch_size
                end_index = min((i+1)*batch_size, nrof_images)
                paths_batch = paths[start_index:end_index]
                images = facenet.load_data(paths_batch, False, False, image_size, do_prewhiten=False)
                # Whats a phase train placeholder??
                feed_dict = { images_placeholder:images }
                emb_array[start_index:end_index,:] = sess.run(embeddings, feed_dict=feed_dict)
            
            return emb_array

<ipython-input-14-281480db3d6c>:9: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset')


In [15]:
embeddings = get_face_embeddings(paths, model_path='./train.pb', mode = "paths")

Loading feature extraction model
Model filename: ./train.pb
embedding size 512
Calculating features for images
Number of epochs 188


In [16]:
tpr, fpr, accuracy, val, val_std, far = lfw.evaluate(embeddings, actual_issame, nrof_folds=10, distance_metric=1, subtract_mean=1)
    
print('Accuracy: %2.5f+-%2.5f' % (np.mean(accuracy), np.std(accuracy)))
print('Validation rate: %2.5f+-%2.5f @ FAR=%2.5f' % (val, val_std, far))
    
auc = metrics.auc(fpr, tpr)
print('Area Under Curve (AUC): %1.3f' % auc)
eer = brentq(lambda x: 1. - x - interpolate.interp1d(fpr, tpr)(x), 0., 1.)
print('Equal Error Rate (EER): %1.3f' % eer)

Accuracy: 0.61483+-0.02084
Validation rate: 0.00267+-0.00389 @ FAR=0.00100
Area Under Curve (AUC): 0.650
Equal Error Rate (EER): 0.390
